In [1]:
from tool import *

## OTD 실적 구하기

In [2]:
# 지난 주 OTD 실적 구하기
thisweek = get_weekname(datetime.date.today()) # 이번주 값 변수 저장
ref_week = get_weekname_from(thisweek, -1) # 지난 주 값 변수 저장
print(ref_week) # 지난주 이름 출력
with open('D:/Data/GSCP raw data.bin', 'rb') as f: # GSCP DB를 불러옴
    df1 = pickle.load(f)

# GSCP DB 에서 검색할 조건 셋팅
c1 = (df1['Ref'] == ref_week) # 지난주로 필터링할 조건
c2 = (df1['Ver'] == 'Latest') # version 명으로 필터링할 조건
c3 = (df1['From Site'] == 'Quanta') # 업체명으로 필터링할 조건
df1 = df1[c1 & c2 & c3] # 상기조건으로 DB 필터링하여 df1으로 저장
df1 = df1[df1['Updated_at'] == df1['Updated_at'].max()] #df1 에서 가장 최근의 update version만 한번더 필터링
df1 = df1[['Mapping Model.Suffix', 'To Site', ref_week]]
df1 = df1[df1[ref_week] > 0].reset_index(drop=True)
df1 = df1.rename(columns={ref_week:'Target'})
df1.insert(0, 'Ref', ref_week)
df1.tail(5)

2023-03-06(W10)


,Ref,Mapping Model.Suffix,To Site,Target
13,2023-03-06(W10),16T90Q-G.AA79B,EEES,15
14,2023-03-06(W10),16T90Q-K.AA75A9,ENCI,40
15,2023-03-06(W10),16T90Q-K.AA75J,EJJP,1
16,2023-03-06(W10),16T90R-G.AA50K,EKHQ,118
17,2023-03-06(W10),16T90R-G.AA76K,EKHQ,1


In [5]:
df1['Target'].sum()

697

In [6]:
with open('D:/Data/Quanta shipment result DB.bin', 'rb') as f:
    sr = pickle.load(f)

In [7]:
if sr[(sr['Week Name'] == ref_week) & (sr['RSD Week Name'] > ref_week)].shape[0] > 0: # 지난 주 선적 실적에 미래구간의 Pre ship이 있다면, 해당 Pre-Ship 값을 지난주 OTD target SP 리스트에 추가하는 로직
    print('There are pre-ship in {}'.format(ref_week))
    add_df = sr[(sr['Week Name'] == ref_week) & (sr['RSD Week Name'] > ref_week)][['Week Name', 'Mapping Model.Suffix', 'Ship To', 'Ship']].reset_index(drop=True)
    add_df.columns = df1.columns
    df1 = pd.concat([df1, add_df]).reset_index(drop=True).convert_dtypes()

There are pre-ship in 2023-03-06(W10)


In [8]:
df1['Target'].sum()

1224.0

In [9]:
sr = sr[sr['Week Name']==ref_week].pivot_table('Ship', index=['Week Name', 'Mapping Model.Suffix', 'Ship To'], aggfunc=sum).reset_index()
sr.columns = ['Ref', 'Mapping Model.Suffix', 'To Site', 'Ship']
sr

,Ref,Mapping Model.Suffix,To Site,Ship
0,2023-03-06(W10),11TG50Q-E.AC10KN,EKHQ,240.0
1,2023-03-06(W10),14T90R-G.AA50K,EKHQ,66.0
2,2023-03-06(W10),14T90R-G.AA56K,EKHQ,24.0
3,2023-03-06(W10),14TD90R-G.AX56K,EKHQ,30.0
4,2023-03-06(W10),15U40Q-G.ARFWK,EKHQ,3.0
5,2023-03-06(W10),15U40Q-G.ARTWK,EKHQ,7.0
6,2023-03-06(W10),15U40R-G.AR30K,EKHQ,35.0
7,2023-03-06(W10),15U40R-G.ARFWK,EKHQ,1.0
8,2023-03-06(W10),15UD40R-G.AX36K,EKHQ,29.0
9,2023-03-06(W10),15UD40R-G.AX56K,EKHQ,12.0


In [10]:
df1 = df1.groupby(['Ref', 'Mapping Model.Suffix', 'To Site']).sum().reset_index()
df1.tail(5)

,Ref,Mapping Model.Suffix,To Site,Target
16,2023-03-06(W10),16T90Q-G.AP76ML,EKHQ,30.0
17,2023-03-06(W10),16T90Q-K.AA75A9,ENCI,40.0
18,2023-03-06(W10),16T90Q-K.AA75J,EJJP,1.0
19,2023-03-06(W10),16T90R-G.AA50K,EKHQ,206.0
20,2023-03-06(W10),16T90R-G.AA76K,EKHQ,1.0


In [11]:
otd_result = pd.merge(df1, sr, how='outer').fillna(0)
otd_result.loc[otd_result['Target'] < otd_result['Ship'], 'Target'] = otd_result.loc[otd_result['Target'] < otd_result['Ship'], 'Ship']
otd_result['Country'] = otd_result['To Site'].replace(site_map)
otd_result['OTD(Result)'] = round(otd_result['Ship'] / otd_result['Target'] * 100, 1)
otd_result = otd_result[['Ref', 'Country', 'Mapping Model.Suffix', 'To Site', 'Target', 'Ship', 'OTD(Result)']].convert_dtypes()
otd_result.insert(1, 'Series', otd_result['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model))
otd_result

,Ref,Series,Country,Mapping Model.Suffix,To Site,Target,Ship,OTD(Result)
0,2023-03-06(W10),11T50Q,Korea,11TG50Q-E.AC10KN,EKHQ,290.0,240,82.8
1,2023-03-06(W10),14T90P,Korea,14T90P-G.ARTGK,EKHQ,49.0,0,0.0
2,2023-03-06(W10),14T90Q,Korea,14T90Q-G.AA79K,EKHQ,1.0,0,0.0
3,2023-03-06(W10),14T90Q,Korea,14T90Q-G.AP79L,EKHQ,6.0,0,0.0
4,2023-03-06(W10),14T90Q,US,14T90Q-K.APG5U1,ENUS_V,5.0,0,0.0
5,2023-03-06(W10),14T90R,Korea,14T90R-G.AA50K,EKHQ,74.0,66,89.2
6,2023-03-06(W10),14T90R,Korea,14T90R-G.AA56K,EKHQ,25.0,24,96.0
7,2023-03-06(W10),14T90R,Korea,14TD90R-G.AX56K,EKHQ,30.0,30,100.0
8,2023-03-06(W10),15U40Q,Korea,15U40Q-G.AR50ML,EKHQ,11.0,0,0.0
9,2023-03-06(W10),15U40Q,Korea,15U40Q-G.ARFWK,EKHQ,8.0,3,37.5


In [12]:
otd_result['Target'].sum()

1324.0

In [13]:
otd_result['Ship'].sum()

1085

In [14]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd_db = pickle.load(f)

In [15]:
otd_db = otd_db[~(otd_db['Ref'] == ref_week)]

In [16]:
otd_db.shape

(627, 9)

In [17]:
otd_db = pd.concat([otd_db, otd_result])

In [18]:
otd_db = otd_db.reset_index(drop=True)

In [19]:
otd_db.shape

(653, 9)

In [20]:
with open('D:/Data/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd_db, f)

In [21]:
with open('D:/Data/DB backup/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd_db, f)

### OTD 실적 보기

In [22]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd = pickle.load(f)

In [23]:
# 주차별 OTD 실적
otd_week = otd.pivot_table(['Target', 'Ship'], index='Ref', aggfunc=sum)
otd_week['OTD(Result)'] = round(otd_week['Ship'] / otd_week['Target'] * 100, 1)
otd_week

,Ship,Target,OTD(Result)
Ref,,,
2022-12-19(W51),1891,6138.0,30.8
2022-12-26(W52),502,4509.0,11.1
2023-01-02(W01),3794,12079.0,31.4
2023-01-09(W02),765,9090.0,8.4
2023-01-16(W03),8535,19400.0,44.0
2023-01-23(W04),1352,12606.0,10.7
2023-01-30(W05),4669,10978.0,42.5
2023-02-06(W06),413,6779.0,6.1
2023-02-13(W07),2224,7886.0,28.2


In [17]:
# 모델별 OTD 실적
otd_model = otd.pivot_table(['Target', 'Ship'], index=['Ref', 'Series'], aggfunc=sum)
otd_model['OTD(Result)'] = round(otd_model['Ship'] / otd_model['Target'] * 100, 1)
otd_model

Ship   Target  OTD(Result)
Ref             Series                            
2022-12-19(W51) 11T50Q   576    630.0         91.4
                14T90Q    15    530.0          2.8
                15U40Q     0   1433.0          0.0
                15U50P  1009   1660.0         60.8
                15U50Q   291   1315.0         22.1
                16T90Q     0    570.0          0.0
2022-12-26(W52) 11T50Q     7     61.0         11.5
                14T90Q   235    515.0         45.6
                15U40Q     0   1433.0          0.0
                15U50P     0    651.0          0.0
                15U50Q   245   1124.0         21.8
                16T90Q    15    725.0          2.1
2023-01-02(W01) 11T50Q    57     57.0        100.0
                14T90Q     5    485.0          1.0
                15U40Q   509   1433.0         35.5
                15U40R    59     59.0        100.0
                15U50P   651    881.0         73.9
                15U50Q  2513   6989.0         36.0
                16T90Q     0   2175.0          0.0
2023-01-09(W02) 14T90Q    95    540.0         17.6
                15U40Q     0    924.0          0.0
                15U50P     0    230.0          0.0
                15U50Q     0   4706.0          0.0
                16T90Q   670   2690.0         24.9
2023-01-16(W03) 11T50Q     0    780.0          0.0
                14T90Q   115    495.0         23.2
                15U40Q   441    964.0         45.7
                15U40R  3376   3586.0         94.1
                15U50P     0    410.0          0.0
                15U50Q  3689  10306.0         35.8
                16T90Q   884   2329.0         38.0
                16T90R    30    530.0          5.7

In [18]:
# 국가별 OTD 실적
otd_country = otd.pivot_table(['Target', 'Ship'], index=['Ref', 'Country'], aggfunc=sum)
otd_country['OTD(Result)'] = round(otd_country['Ship'] / otd_country['Target'] * 100, 1)
otd_country

Ship   Target  OTD(Result)
Ref             Country                               
2022-12-19(W51) Germany        0     20.0          0.0
                Korea       1876   5648.0         33.2
                Poland         0     10.0          0.0
                Singapore     15     50.0         30.0
                Spain          0     40.0          0.0
                US             0    370.0          0.0
2022-12-26(W52) Austrailia     0     15.0          0.0
                Germany        5     80.0          6.2
                Korea        412   3949.0         10.4
                Poland         0     10.0          0.0
                Singapore      0     35.0          0.0
                Spain         20     40.0         50.0
                Taiwan         0     10.0          0.0
                US            65    370.0         17.6
2023-01-02(W01) Austrailia     0     15.0          0.0
                Canada         0     20.0          0.0
                Germany        0     75.0          0.0
                Korea       3794  11189.0         33.9
                Poland         0     10.0          0.0
                Singapore      0     35.0          0.0
                Spain          0     40.0          0.0
                Taiwan         0     10.0          0.0
                US             0    685.0          0.0
2023-01-09(W02) Austrailia    45     25.0        180.0
                Canada        40     40.0        100.0
                Germany        0    140.0          0.0
                Korea          0   7685.0          0.0
                Poland        10     10.0        100.0
                Singapore     35     35.0        100.0
                Spain         40     40.0        100.0
                Taiwan        10     10.0        100.0
                UK             5     15.0         33.3
                US           580   1090.0         53.2
2023-01-16(W03) Germany       90    205.0         43.9
                Korea       8115  18625.0         43.6
                UK            10     10.0        100.0
                US           320    560.0         57.1

### OTD 실적 수정

In [18]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd = pickle.load(f)

In [21]:
otd.shape

(627, 8)

In [22]:
otd = otd[~(otd['Ref'] == get_weekname_from(get_weekname(datetime.date.today()), -1))]

In [23]:
otd.shape

(581, 8)

In [24]:
with open('D:/Data/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd, f)

##  OTD 차질 사유 업데이트

In [2]:
with open('D:/Data/OTD_result_db.bin', 'rb') as f:
    otd = pickle.load(f)

In [3]:
week = '2023-02-13(W07)'

In [5]:
otd[otd['Ref'] == week]['Target'].sum()

7886.0

In [6]:
otd[otd['Ref'] == week]['Ship'].sum()

2224

In [17]:
reason_df = pd.read_clipboard()
reason_df = reason_df[['Ref', 'Mapping Model.Suffix', 'To Site', 'Target', 'Ship', 'OTD Failure Reason']]
reason_df

,Ref,Mapping Model.Suffix,To Site,Target,Ship,OTD Failure Reason
0,2023-02-13(W07),11TG50Q-E.AC10KN,EKHQ,780,0,Input 350pcs，camera 2/24*470
1,2023-02-13(W07),14T90P-G.ARTGK,EKHQ,49,0,2/17 issue PO
2,2023-02-13(W07),14T90Q-G.AA79K,EKHQ,50,0,2/21 Input
3,2023-02-13(W07),14T90Q-G.AP50ML,EKHQ,70,0,"shortage TOP,Cover"
4,2023-02-13(W07),14T90Q-G.AP56ML,EKHQ,100,50,2/19 Air
5,2023-02-13(W07),14T90Q-G.AP79L,EKHQ,60,0,"shortage TOP,Cover"
6,2023-02-13(W07),14T90Q-G.AR5GK,EKHQ,40,0,2/21 Input
7,2023-02-13(W07),14T90Q-G.AS50ML,EKHQ,40,0,"shortage TOP,Cover"
8,2023-02-13(W07),14T90Q-K.AA77A1,EEUK,10,0,2/21 Input
9,2023-02-13(W07),14T90Q-K.AAB9U1,ENUS_V,50,39,WIP


In [21]:
otd = otd.merge(reason_df, how='outer')

In [22]:
otd.head()

,Ref,Series,Country,Mapping Model.Suffix,To Site,Target,Ship,OTD(Result),OTD Failure Reason
0,2022-12-19(W51),11T50Q,Korea,11TG50Q-E.AC10KN,EKHQ,630.0,576,91.4,NaN
1,2022-12-19(W51),14T90Q,Singapore,14T90Q-G.AA55A3,EASL,5.0,5,100.0,NaN
2,2022-12-19(W51),14T90Q,Korea,14T90Q-G.AA56K,EKHQ,80.0,0,0.0,NaN
3,2022-12-19(W51),14T90Q,Singapore,14T90Q-G.AA75A3,EASL,25.0,10,40.0,NaN
4,2022-12-19(W51),14T90Q,Spain,14T90Q-G.AA78B,EEES,40.0,0,0.0,NaN


In [23]:
with open('D:/Data/OTD_result_db.bin', 'wb') as f:
    pickle.dump(otd, f)